# Proyecto Final 
## Curso de Teoría de la Simulación
### Impartido por ing. Uayeb Caballero

Proyecto desarrollado por: 
- Jeimie Vanesa Herrera
- Kattherine Mayely Hernandez

Consistente en la simulación del funcionamiento de un banco, donde el cliente llega con la idea del tipo de servicio que desea recibir, este puede ser:
- ATM (Cajero automático)
- Servicio personalizado: Caja o Servicio al cliente

### 1. Importamos librerías

In [71]:
import pandas as pd
import numpy as np
import simpy
import random

### 2. Definimos la clase Banco que representará los recursos disponibles para los clientes (ATM, cajas y agentes de servicio)

In [72]:
class Banco:
    def __init__(self, env, num_atms, num_cajas, num_agentes):
        self.env = env  # Entorno de simulación de SimPy (maneja el tiempo y eventos)

        # Definimos los recursos del banco:
        # Cada uno es un recurso limitado que puede ser solicitado por los clientes
        self.atms = simpy.Resource(env, capacity=num_atms)  # Cajeros automáticos (ATM)
        self.cajas = simpy.Resource(env, capacity=num_cajas)  # Cajas presenciales con personal
        self.servicio_cliente = simpy.Resource(env, capacity=num_agentes)  # Agentes para atención personalizada

Definimos la estructura de los tipos de servicio y sus operaciones.

In [73]:
SERVICIOS = {
    "ATM": ["Deposito", "Retiro"],
    "caja": ["Deposito", "Retiro"],
    "servicio_cliente": ["Consulta", "Apertura de cuenta"]
}

### 3. Definimos la clase que representa a un cliente en el banco

In [74]:
class Cliente:
    def __init__(self, env, id, banco):
        self.env = env                  # Entorno de simulación SimPy
        self.id = id                    # ID del cliente
        self.banco = banco              # Referencia al banco (con recursos: atms, cajas, servicio_cliente)
        self.servicio_personalizado = random.choice([True, False])  # ¿Usará atención personalizada o ATM?
        self.tipo_servicio = None      # Tipo de servicio que usará (caja, servicio_cliente, ATM)
        self.operacion = None          # Tipo de operación que desea hacer
        self.eventos = pd.DataFrame()  # DataFrame donde se registran los eventos del cliente

        # Se define el tipo de servicio y operación que realizará
        self._definir_servicio()

        # Se inicia el proceso del cliente en el entorno de simulación
        self.env.process(self.start())

    # Método interno para definir tipo de servicio y operación según si es personalizado o no
    def _definir_servicio(self):
        if self.servicio_personalizado:
            # Escoge aleatoriamente entre las claves del diccionario SERVICIOS
            self.tipo_servicio = random.choice(list(SERVICIOS.keys()))
        else:
            # Si no requiere atención personalizada, solo usa ATM
            self.tipo_servicio = "ATM"
    
    # Escoge una operación aleatoria del tipo de servicio seleccionado
        self.operacion = random.choice(SERVICIOS[self.tipo_servicio])

    # Registra un evento en el DataFrame con información del cliente, tiempo y operación
    def registrar_evento(self, evento):
        self.eventos = pd.concat([self.eventos, pd.DataFrame({
            "id": [self.id],
            "evento": [evento],
            "time": [self.env.now],
            "servicio": [self.tipo_servicio],
            "operacion": [self.operacion]
        })], ignore_index=True)

    # Método principal que ejecuta el comportamiento del cliente dentro del banco
    def start(self):
        yield self.env.timeout(0)  # Necesario para iniciar el proceso en SimPy

        # Momento en el que el cliente llega al banco
        llegada = self.env.now
        print(f"{llegada:.2f}: Cliente {self.id} llegó - hará cola para {self.tipo_servicio}.")
        self.registrar_evento("LLEGÓ")

        # Se determina qué recurso usará según el tipo de servicio
        if self.tipo_servicio == "ATM":
            recurso = self.banco.atms
            t_min, t_max = 1, 3          # Tiempo de atención en ATM
        elif self.tipo_servicio == "caja":
            recurso = self.banco.cajas
            t_min, t_max = 3, 6          # Tiempo en caja
        else:
            recurso = self.banco.servicio_cliente
            t_min, t_max = 6, 10         # Tiempo con agente de servicio al cliente

        # Solicita acceso al recurso y espera a que lo atiendan
        with recurso.request() as req:
            yield req  # Espera turno
            print(f"{self.env.now:.2f}: Cliente {self.id} siendo atendido ({self.tipo_servicio}) - '{self.operacion}'")
            self.registrar_evento("ATENDIDO")

            # Tiempo que tarda la atención
            duracion = random.randint(t_min, t_max)
            yield self.env.timeout(duracion)

            # Atención finalizada
            print(f"{self.env.now:.2f}: Cliente {self.id} finalizó ({self.tipo_servicio}) - '{self.operacion}'")
            self.registrar_evento("FINALIZADO")


### 4. Definimos la clase que representa una simulación del movimiento en el banco

In [75]:
class Simulacion:
    def __init__(self, env, num_clientes, num_atms, num_cajas, num_agentes):
        self.env = env
        self.banco = Banco(env, num_atms, num_cajas, num_agentes)  # Se crea una instancia del banco con sus recursos
        self.clientes = []  # Lista para almacenar todos los clientes que serán generados

        # Se generan los clientes
        for i in range(num_clientes):
            # El tiempo entre llegadas sigue una distribución exponencial con media 9
            llegada = random.expovariate(1 / 9)
            # Se inicia el proceso de lanzamiento del cliente con un retraso (delay)
            env.process(self.lanzar_cliente(i, llegada))

    # Método que representa la llegada de un cliente tras cierto tiempo (delay)
    def lanzar_cliente(self, id, delay):
        yield self.env.timeout(delay)  # Espera el tiempo de llegada
        cliente = Cliente(self.env, id, self.banco)  # Crea un nuevo cliente
        self.clientes.append(cliente)  # Agrega el cliente a la lista


### 5. Iniciar la simulación

In [76]:
# Crear el entorno de simulación de SimPy
env = simpy.Environment()

# Establecer la semilla para el generador de números aleatorios para hacer la simulación reproducible
random.seed(111)

# Definir la cantidad de clientes y recursos disponibles en el banco
num_clientes = 30  # Número de clientes que se simularán
num_atms = 2  # Número de cajeros automáticos disponibles
num_cajas = 4  # Número de cajas disponibles para atender clientes
num_agentes = 3  # Número de agentes en el servicio al cliente

# Crear la instancia de la simulación, pasando el entorno y los parámetros definidos
sim = Simulacion(env, num_clientes, num_atms, num_cajas, num_agentes)

# Ejecutar la simulación hasta un tiempo determinado (480 minutos, equivalentes a 8 horas)
env.run(until=60 * 8)


0.94: Cliente 27 llegó - hará cola para servicio_cliente.
0.94: Cliente 27 siendo atendido (servicio_cliente) - 'Consulta'
1.03: Cliente 26 llegó - hará cola para ATM.
1.03: Cliente 26 siendo atendido (ATM) - 'Retiro'
1.67: Cliente 6 llegó - hará cola para ATM.
1.67: Cliente 6 siendo atendido (ATM) - 'Retiro'
1.94: Cliente 7 llegó - hará cola para servicio_cliente.
1.94: Cliente 7 siendo atendido (servicio_cliente) - 'Consulta'
2.15: Cliente 1 llegó - hará cola para caja.
2.15: Cliente 1 siendo atendido (caja) - 'Retiro'
2.18: Cliente 29 llegó - hará cola para ATM.
2.25: Cliente 14 llegó - hará cola para ATM.
2.35: Cliente 9 llegó - hará cola para servicio_cliente.
2.35: Cliente 9 siendo atendido (servicio_cliente) - 'Apertura de cuenta'
2.46: Cliente 21 llegó - hará cola para ATM.
2.93: Cliente 13 llegó - hará cola para ATM.
3.38: Cliente 19 llegó - hará cola para ATM.
3.67: Cliente 6 finalizó (ATM) - 'Retiro'
3.67: Cliente 29 siendo atendido (ATM) - 'Deposito'
3.88: Cliente 17 llegó 

### Unir todos los dataframes de eventos de los clientes para visualizar todos los resultados

In [77]:
todos_los_eventos = pd.concat([cliente.eventos for cliente in sim.clientes], ignore_index=True)

todos_los_eventos 

,id,evento,time,servicio,operacion
0,27,LLEGÓ,0.941225,servicio_cliente,Consulta
1,27,ATENDIDO,0.941225,servicio_cliente,Consulta
2,27,FINALIZADO,6.941225,servicio_cliente,Consulta
3,26,LLEGÓ,1.026502,ATM,Retiro
4,26,ATENDIDO,1.026502,ATM,Retiro
...,...,...,...,...,...
85,16,ATENDIDO,20.955940,ATM,Retiro
86,16,FINALIZADO,21.955940,ATM,Retiro
87,2,LLEGÓ,25.706777,caja,Deposito
88,2,ATENDIDO,25.706777,caja,Deposito
